In [3]:
import glob
from datetime import datetime 
from bs4 import BeautifulSoup
import pandas as pd


INPUT_DATA_FOLDER = '0_isw_data'
OUTPUT_FOLDER = '1_isw_prepared'
OUTPUT_DATA_FILE = 'isw_all_days.csv'

by_dates = glob.glob(f'{INPUT_DATA_FOLDER}/*.html')

In [4]:
all_data = []

for i in by_dates:
    
    d={}
    
    
    file_name = i.split('\\')[-1].split('__')
    date = datetime.strptime(file_name[0], '%B_%d_%Y')
    url = file_name[1].split('.')[0]
    
    
    with open(i, 'r', encoding="utf-8", errors='namereplace') as cfile:
        parsed_html = BeautifulSoup(cfile, 'html.parser')
        try:
            title = parsed_html.head.find('title').text
        except AttributeError:
            title = ""
        try:
            link = parsed_html.head.find('link', attrs={'rel':"canonical"}, href = True).attrs["href"]
        except (AttributeError, KeyError):
            link = ""
        try:
            text_title = parsed_html.body.find('h1', attrs={'id':'page-title'}).text
        except AttributeError:
            text_title = ""
        try:
            text_main = parsed_html.body.find('div', attrs={'class':'field field-name-body field-type-text-with-summary field-label-hidden'})
        except AttributeError:
            text_main = ""
       
        dictionary = {
            'date':date,
            'short_url':url,
            'title':title,
            'text_title':text_title,
            'full_url':link,
            'main_html':text_main
        }
        
        
        all_data.append(dictionary)

In [5]:
df = pd.DataFrame.from_dict(all_data)

df = df.sort_values(by = ['date'])

df.head()

,date,short_url,title,text_title,full_url,main_html
92,2022-02-24,russia_ukraine_warning_update_initial_russian_...,Russia-Ukraine Warning Update: Initial Russian...,Russia-Ukraine Warning Update: Initial Russian...,/backgrounder/russia-ukraine-warning-update-in...,"[[[ , <p align=""center"" style=""text-align: lef..."
93,2022-02-25,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"[[[ , <p align=""center"" style=""text-align: lef..."
94,2022-02-27,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"[[[ , <p style=""text-align: left;""><strong><sp..."
95,2022-02-28,russian_offensive_campaign_assessment_february...,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef..."
181,2022-03-01,russian_offensive_campaign_assessment_march_1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef..."


In [7]:
df.to_csv(f'{OUTPUT_FOLDER}/{OUTPUT_DATA_FILE}', sep=';', index = False)